In [ ]:
import scipy.io
import numpy as np
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Conv2D, MaxPool2D, Flatten
from keras import optimizers
import keras.utils as np_utils
from keras import backend as K
import matplotlib.pyplot as plt
import sys
import urllib
import os


####### download mnist file from website 
if not os.path.isfile("train-images-idx3-ubyte.gz"):
    urllib.request.urlretrieve("http://yann.lecun.com/exdb/mnist/train-images-idx3-ubyte.gz","train-images-idx3-ubyte.gz")
if not os.path.isfile("train-labels-idx1-ubyte.gz"):
    urllib.request.urlretrieve("http://yann.lecun.com/exdb/mnist/train-labels-idx1-ubyte.gz","train-labels-idx1-ubyte.gz")

######## reading from the .gz files
import gzip
f=gzip.open("train-images-idx3-ubyte.gz",'rb')
buf=f.read(4)
magic_num=np.frombuffer(buf,dtype=np.dtype(np.int32).newbyteorder('>'))
if magic_num != 2051:
    print("invalid file ... exiting...")
    exit()
buf=f.read(4)
N=np.frombuffer(buf,dtype=np.dtype(np.int32).newbyteorder('>'))[0]
buf=f.read(4)
H=np.frombuffer(buf,dtype=np.dtype(np.int32).newbyteorder('>'))[0]
buf=f.read(4)
W=np.frombuffer(buf,dtype=np.dtype(np.int32).newbyteorder('>'))[0]
train_img=np.frombuffer(f.read(H*W*N),dtype=np.uint8)
train_img=train_data.reshape(N,H,W,1)
f.close()

f=gzip.open("train-labels-idx1-ubyte.gz",'rb')
buf=f.read(4)
magic_num=np.frombuffer(buf,dtype=np.dtype(np.int32).newbyteorder('>'))
if magic_num != 2049:
    print("invalid file ... exiting...")
    exit()
buf=f.read(4)
N=np.frombuffer(buf,dtype=np.dtype(np.int32).newbyteorder('>'))[0]
train_label=np.frombuffer(f.read(N),dtype=np.uint8)
train_label=train_label.reshape(N,1)
f.close()



if K.image_data_format() == 'channels_last':
    train_img = train_img.reshape(train_img.shape[0],train_img.shape[1], train_img.shape[2], train_img.shape[3])
else:
    train_img = train_img.reshape(train_img.shape[0], 1, train_img.shape[1], train_img.shape[2])
print(train_img.shape)


def new_model():
    model=Sequential()
    model.add(Conv2D(filters=32,kernel_size=(5,5),padding='Valid',activation='relu',input_shape=(28,28,1)));
    model.add(MaxPool2D(pool_size=(2,2)))
    model.add(Conv2D(filters=12,kernel_size=(5,5),padding='Valid',activation='relu'));
    model.add(MaxPool2D(pool_size=(2,2)))
    model.add(Flatten())
    model.add(Dense(10,activation='softmax'))
    return model

def list_sum_vec(a,b):
    for i in range(len(a)):
        a[i]=a[i]+b
    return a
def list_mul_vec(a,b):
    for i in range(len(a)):
        a[i]=a[i]*b
    return a
def list_sub_vec(a,b):
    for i in range(len(a)):
        a[i]=a[i]-b
    return a
def list_div_vec(a,b):
    for i in range(len(a)):
        a[i]=a[i]/b
    return a
def list_sum_list(a,b):
    for i in range(len(a)):
        a[i]=a[i]+b[i]
    return a
def list_mul_list(a,b):
    for i in range(len(a)):
        a[i]=np.multiply(a[i],b[i])
    return a
def list_sub_list(a,b):
    for i in range(len(a)):
        a[i]=a[i]-b[i]
    return a
def list_div_list(a,b):
    for i in range(len(a)):
        a[i]=np.divide(a[i],b[i])
    return a
def list_rand(a):
    for i in range(len(a)):
        a[i]=-1+np.random.random(a[i].shape)*2
    return a

model=new_model();
sgd=optimizers.SGD(lr=0.01,decay=0.0001,momentum=0.9)
model.compile(optimizer=sgd,loss='mean_squared_error',metrics=['accuracy'])
model.fit(train_img,train_label,batch_size=1000,epochs=1,verbose=1)


pop=[];vel=[];
iter_c=1
pop_sz=10
for i in range(10):
    m=new_model()
    pop.append(list_rand(m.get_weights()));
    m=new_model()
    vel.append(list_rand(m.get_weights()));

obj=[];
for i in range(10):
    model.set_weights(pop[i])
    v=model.evaluate(train_img,train_label);
    obj.append(v[1])
    


max_iter=100
p=np.argmax(obj)
g_best_pop=pop[p].copy()
g_best_obj=obj[p].copy()
l_best_pop=pop.copy()
l_best_obj=obj.copy()
iter_c=1
plot_mean_obj=[]
plot_obj=[]
while iter_c<max_iter:
    for i in range(pop_sz):
        t_p=list_mul_vec(list_sub_list(l_best_pop[i],pop[i]),(np.random.rand(1)*0.2))
        t_p=list_sum_list(t_p,list_mul_vec(list_sub_list(g_best_pop,pop[i]),(np.random.rand(1)*0.2)))
        vel[i]=list_sum_list(vel[i],t_p)
        pop[i]=list_sum_list(pop[i],vel[i])
        model.set_weights(pop[i])
        v=model.evaluate(train_img,train_label,verbose=0,batch_size=1000)
        obj[i]=v[1]
        if obj[i]>l_best_obj[i]:
            l_best_pop[i]=pop[i].copy()
            l_best_obj[i]=obj[i].copy()
            
        if obj[i]>g_best_obj:
            g_best_pop=pop[i].copy()
            g_best_obj=obj[i].copy()
        
    plot_obj.append(g_best_obj)
    plot_mean_obj.append(np.mean(l_best_obj))
    iter_c=iter_c+1
    print('[{0}] {1}%'.format('#'*(iter_c//(max_iter//20)),iter_c),end='\r')
    sys.stdout.flush()

import matplotlib.pyplot as plt
plt.plot(plot_mean_obj)
plt.show()